In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
# from gpt_index.composability.joint_qa_summary import QASummaryGraphBuilder
from gpt_index import SimpleDirectoryReader, ServiceContext, LLMPredictor
from gpt_index import GPTSimpleVectorIndex, GPTListIndex, GPTSimpleKeywordTableIndex
from gpt_index.composability import ComposableGraph
from langchain.chat_models import ChatOpenAI
from gpt_index.response.notebook_utils import display_response

#### Load Documents

In [ ]:
reader = SimpleDirectoryReader('../paul_graham_essay/data')
documents = reader.load_data()

#### Parse into Nodes

In [ ]:
from gpt_index.node_parser import SimpleNodeParser
nodes = SimpleNodeParser().get_nodes_from_documents(documents)

#### Add to Docstore

In [ ]:
MONGO_URI = ""

In [ ]:
from gpt_index.docstore import MongoDocumentStore
docstore = MongoDocumentStore.from_uri(uri=MONGO_URI)

In [ ]:
docstore.add_documents(nodes)

#### Define Multiple Indexes

Each index uses the same underlying Node.

In [ ]:
list_index = GPTListIndex(nodes, docstore=docstore)

In [42]:
list_index.save_to_dict()

AttributeError: 'MongoDocumentStore' object has no attribute 'to_dict'

In [ ]:
vector_index = GPTSimpleVectorIndex(nodes, docstore=docstore) 

In [ ]:
keyword_table_index = GPTSimpleKeywordTableIndex(nodes, docstore=docstore) 

In [ ]:
# NOTE: the docstore still has the same nodes
len(docstore.docs)

#### Test out some Queries

In [ ]:
llm_predictor_chatgpt = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context_chatgpt = ServiceContext.from_defaults(llm_predictor=llm_predictor_chatgpt, chunk_size_limit=1024)

In [ ]:
list_response = list_index.query("What is a summary of this document?") 

In [ ]:
display_response(list_response)

In [ ]:
vector_response = vector_index.query("What did the author do growing up?") 

In [ ]:
display_response(vector_response)

In [ ]:
keyword_response = keyword_table_index.query("What did the author do after his time at YC?")

In [ ]:
display_response(keyword_response)